---

<center>

# **Uncertainty Quantification**

<center>

---


## Static Problem:

---

So far, in [first](01_Intro.ipynb) and [second](02_Probability_Theory_Basics.ipynb) notebooks we introduced the basic steps to follow in the process of stochastic thinking and uncertainty quantification. We also tackled some probability theory basics in order to grasp the fundamentals.

Now we are going to analyze an **elastic system on the static regime**. As the *elastic* word assumes, we are going to be working under **the hypothesis of lineality, i.e Hooke's law**. 

Then the relation between the displacement $d$ and the applied force  $f$ is proportional by an elastic constant $k$

\begin{equation}
\tag{3.1}
f=kd
\end{equation}

this constant $k$ is the stiffness of the material.

Consider a mechanical test as modeled by *Fig 3.1*, suppose that the model can be accurately described by Eq. 3.1. Additionaly the test resulted in two sets of data:

* Displacement: ${d(t_1), d(t_2),...,d(t_N)}$
* Force: ${F^{exp}(t_1),F^{exp}(t_2),...,F^{exp}(t_N)}$

<center> <img src="diagrams\03_mechanical_test.svg" width="340" height="400"> 

*Fig 3.1: mechanical test model*

</center>

Now, as we said in [notebook 1](01_Intro.ipynb), all measurements end up being uncertain variables, i.e the **data has noise**. This way, with the purpose of highlighting this effect, we consider that the relation between the measured force $F^{exp}$ and the displacement $d$ can be represented with some **additive error $\nu$**

\begin{equation}
\tag{3.2}
F^{exp}=kd+\nu
\end{equation}

this $\nu$ is a random variable that describe the measurement error. In particular consider that $\nu$ is a random variable with a **normal distribution** with $\mu=0$ and $\sigma_{\nu}^2$, simply defined as $\nu \sim N(0,\sigma_{\nu}^2)$.

In [108]:
import matplotlib.pyplot as plt
import numpy as np

N = 5
mu = 0
sigma = 4
k_true = 10

d = np.linspace(1, N, N)
nu = np.random.normal(mu, sigma, N)

f_true = k_true * d
f_exp = k_true * d + nu

plt.ioff()
fig, axis = plt.subplots(figsize=(11, 7))
plt.plot(d, f_true, linewidth=2, color='blue', label=f'True line (k = {k_true})')
plt.scatter(d, f_exp, color='red', label='Measurements', facecolor='none', edgecolor='red', linewidth=2)
axis.set_title('Experiment vs reference', fontsize=14, pad=10, fontweight='bold')
axis.set_ylim(0, 60)
axis.set_xlim(0, 6)
axis.set_xlabel('Displacement (d)', fontsize=12, fontweight='bold')
axis.set_ylabel('Force (f)', fontsize=12, fontweight='bold')
axis.set_aspect(0.1)
axis.grid(linestyle='--', alpha=0.5)
axis.legend(loc='upper left')
plt.savefig('diagrams/03_experiment_vs_reference.svg', bbox_inches='tight', pad_inches=0.4, dpi=300)

With Python the experiment was simulated as if we extracted 5 data points of force `f_exp` and displacement `d`, Fig. 3.2 shows the measurements of the experiment, given by $F^{exp}=kd+\nu$ with $\nu \sim N(0,4)$, vs the reference model ($f=kd$).

<center> <img src="diagrams\03_experiment_vs_reference.svg" width="700" height="500"> 

*Fig 3.2: reference vs experiment*

</center>

Now, in relation to a mechanical test like the one in Fig. 3.1, an engineering team can have as objective determinating the stiffness $k$ of some mechanichal component. This search might be driven because the team does not know the material that is it made of or because some other reason that makes $k$ difficult to know.

The value of the stiffness $k$ will be estimated from the data ${d(t_1), d(t_2),...,d(t_N)}$ and ${F^{exp}(t_1),F^{exp}(t_2),...,F^{exp}(t_N)}$ using the [**least squares method**](note_least_squares.ipynb). This strategy is deterministic and the estimation obtained is a **point estimation**. For that consider Eq. 3.1 in **vector form**

$$
\begin{bmatrix}
d_1 \\
d_2 \\
\vdots \\
d_N
\end{bmatrix}
k =
\begin{bmatrix}
F_1^{\text{exp}} \\
F_2^{\text{exp}} \\
\vdots \\
F_N^{\text{exp}}
\end{bmatrix}
$$

this can be written as

\begin{equation}
\tag{3.3}
[A]k=\mathbf{F}^{exp}
\end{equation}

Now, solving via the least squares method

\begin{equation}
\tag{3.4}
\hat{k}=([A]^T[A])^{-1}[A]^T\mathbf{F}^{exp}
\end{equation}

notice that $\hat{k}$ is used to define the **parameter estimation** for $k$.

In [106]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import lstsq

N = 5
mu = 0
sigma = 4
k_true = 10

d = np.linspace(1, N, N).reshape(-1, 1)  # Column vector for lstsq function
nu = np.random.normal(mu, sigma, N).reshape(-1, 1)  # Reshape noise to column vector

f_true = k_true * d
f_exp = k_true * d + nu

k_hat = lstsq(d, f_exp)[0][0]  # Extract the solution only from lstsq function

f_identified = k_hat * d

# print(f"True k: {k_true}")
# print(f"Estimated k: {k_hat}")

plt.ioff()
fig, axis = plt.subplots(figsize=(11, 7))
plt.scatter(d, f_exp, color='red', label='Measurements', facecolor='none', edgecolor='red', linewidth=2)
plt.plot(d, f_identified, color='magenta', linewidth=2, label=f'Fitted line (k = {k_hat[0]:.2f})')
plt.plot(d, f_true, linewidth=1.75, linestyle='--',color='blue', label=f'True line (k = {k_true})')
axis.set_title(r'Identified model with $\hat{k}$', fontsize=14, pad=10, fontweight='bold')
axis.set_ylim(0, 60)
axis.set_xlim(0, 6)
axis.set_xlabel('Displacement (d)', fontsize=12, fontweight='bold')
axis.set_ylabel('Force (f)', fontsize=12, fontweight='bold')
axis.legend(loc='upper left')
axis.set_aspect(0.1)
plt.grid(linestyle='--', alpha=0.5)
plt.savefig('diagrams/03_identified_model.svg', bbox_inches='tight', pad_inches=0.4, dpi=300)


The least-squares method approach here is easily applied with `scipy`'s least-squares solver `lstsq`, Fig. 3.3 shows the experimental data, the true line as reference and the **least-squares approximation**. The method, for this data set, gives us a value for $\hat{k}$ of $9.09$ in contrast with $10$ as the real value.

<center> <img src="diagrams\03_identified_model.svg" width="700" height="500"> 

*Fig 3.3: reference vs experiment vs least-squares model*

</center>

Notice that we only used 5 data point but is important to highlight that the bigger the data set the better the least-squares method performs. For example, for a data set of 50 points we have an error of around $\pm 0.02$ and for 100 data points around $\pm 0.002$.

Just to let the reader know, if we see that the experimental data can be approximated using non-linear models, for example a quadratic one $k_2d^2+k_1d$ or cubic like $k_3d^3+k_1d$, tools like `scipy`'s least-squares solver `lstsq` (or others) can solve for the desired parameters ($k_i$) if needed.